In [1]:
# pip install numpy
# pip install pandas
# pip install pandas-gbq
# pip install google-auth
# pip install cryptography

In [4]:
import numpy as np
import pandas as pd

from google.oauth2.service_account import Credentials
from cryptography.fernet import Fernet

In [5]:
df = pd.read_csv("4g_daily.csv")
df.head()

,AREA,DATE_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC Setup Success Rate (Service),ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra-Frequency Handover Out Success Rate,...,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,pmCellDownTimeAuto,pmCellDownTimeMan,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL
0,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML1,0.0,99.99393,99.98195,99.97589,0.06679,99.66047,...,2.59416,11.543,0.54444,0.00426,0.00255,23,0,3.14533,0.00322,0.01026
1,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML2,0.0,99.88980,99.91135,99.80125,0.09515,99.41480,...,1.88985,10.274,1.47361,0.01188,0.00950,14,0,3.54175,0.00866,0.07735
2,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML3,0.0,99.97177,99.93837,99.91015,0.06221,99.11193,...,2.11394,10.925,0.18611,0.00181,0.00121,16,0,3.38281,0.00714,0.37989
3,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT1,0.0,99.57377,99.46474,99.04079,0.24718,99.87531,...,2.04409,11.349,0.34167,0.00284,0.00172,11,0,3.44817,0.03223,0.01450
4,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT2,0.0,99.24931,99.09149,98.34761,0.44175,99.49495,...,1.60831,10.997,0.18333,0.00147,0.00126,10,0,3.87995,0.02036,0.02991


In [2]:
df_hourly = pd.read_csv("4g_hourly_dummy.csv")
df_hourly.head()

,AREA,DATE_ID,HOUR_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC Setup Success Rate (Service),ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,...,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,Band_,Sector_,NEID_,SiteID_
0,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME1,0,100.0000,100.00000,100.00000,0.0,...,-111.24809,2.58517,11.558,0,0,0,L2300,1,SAA108ME1,SAA108
1,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME2,0,100.0000,100.00000,100.00000,0.0,...,-113.47277,3.8755,12.635,0,0,0,L2300,2,SAA108ME1,SAA108
2,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME3,0,0.0000,0.00000,0.00000,0.0,...,\N,\N,\N,0,0,0,L2300,3,SAA108ME1,SAA108
3,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF1,0,99.9239,99.87849,99.80248,0.0,...,-109.93618,3.10492,12.473,0,0,0,L2300,1,SAA108ME1,SAA108
4,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF2,0,100.0000,100.00000,100.00000,0.0,...,-113.14835,2.75921,12.306,0,0,0,L2300,2,SAA108ME1,SAA108


In [4]:
df.columns = df.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True).str.replace("-", "_")

df["DATE_ID"] = pd.to_datetime(df["DATE_ID"])

for i in ["DL_Resource_Block_Utilizing_Rate", "UL_Resource_Block_Utilizing_Rate", "inter_freq_HO", "UL_RSSI_dbm", "SE_2", "Average_CQI_nonHOME"]:
    df[i] = pd.to_numeric(df[i], errors="coerce")

df.head()

,AREA,DATE_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC_Setup_Success_Rate_Service,ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra_Frequency_Handover_Out_Success_Rate,...,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,pmCellDownTimeAuto,pmCellDownTimeMan,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL
0,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML1,0.0,99.99393,99.98195,99.97589,0.06679,99.66047,...,2.59416,11.543,0.54444,0.00426,0.00255,23,0,3.14533,0.00322,0.01026
1,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML2,0.0,99.88980,99.91135,99.80125,0.09515,99.41480,...,1.88985,10.274,1.47361,0.01188,0.00950,14,0,3.54175,0.00866,0.07735
2,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML3,0.0,99.97177,99.93837,99.91015,0.06221,99.11193,...,2.11394,10.925,0.18611,0.00181,0.00121,16,0,3.38281,0.00714,0.37989
3,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT1,0.0,99.57377,99.46474,99.04079,0.24718,99.87531,...,2.04409,11.349,0.34167,0.00284,0.00172,11,0,3.44817,0.03223,0.01450
4,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT2,0.0,99.24931,99.09149,98.34761,0.44175,99.49495,...,1.60831,10.997,0.18333,0.00147,0.00126,10,0,3.87995,0.02036,0.02991


In [16]:
df_hourly.columns = df_hourly.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True).str.replace("-", "_")

df_hourly["DATE_ID"] = pd.to_datetime(df_hourly["DATE_ID"])

for i in ["DL_Resource_Block_Utilizing_Rate", "UL_Resource_Block_Utilizing_Rate", "inter_freq_HO", "UL_RSSI_dbm", "SE_2", "Average_CQI_nonHOME", "volte_traffic_erl", "volte_payload_DL", "volte_payload_UL"]:
    df_hourly[i] = pd.to_numeric(df_hourly[i], errors="coerce")

df_hourly.head()

,AREA,DATE_ID,HOUR_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC_Setup_Success_Rate_Service,ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,...,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,Band_,Sector_,NEID_,SiteID_
0,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME1,0,100.0000,100.00000,100.00000,0.0,...,-111.24809,2.58517,11.558,0.0,0.0,0.0,L2300,1,SAA108ME1,SAA108
1,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME2,0,100.0000,100.00000,100.00000,0.0,...,-113.47277,3.87550,12.635,0.0,0.0,0.0,L2300,2,SAA108ME1,SAA108
2,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME3,0,0.0000,0.00000,0.00000,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,L2300,3,SAA108ME1,SAA108
3,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF1,0,99.9239,99.87849,99.80248,0.0,...,-109.93618,3.10492,12.473,0.0,0.0,0.0,L2300,1,SAA108ME1,SAA108
4,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF2,0,100.0000,100.00000,100.00000,0.0,...,-113.14835,2.75921,12.306,0.0,0.0,0.0,L2300,2,SAA108ME1,SAA108


In [5]:
def dtype_to_schema(dtype):
    if dtype == "object":
        return "STRING"
    elif dtype == "int64":
        return "INT64"
    elif dtype == "float64":
        return "FLOAT64"
    elif "datetime" in str(dtype):
        return "DATE"
    else:
        return "UNKNOWN"

schema = [{"name": col, "type": dtype_to_schema(df[col].dtype)} for col in df.columns]
schema

[{'name': 'AREA', 'type': 'STRING'},
 {'name': 'DATE_ID', 'type': 'DATE'},
 {'name': 'ERBS', 'type': 'STRING'},
 {'name': 'EUTRANCELLFDD', 'type': 'STRING'},
 {'name': 'LTE_CSFB_SR', 'type': 'FLOAT64'},
 {'name': 'RRC_Setup_Success_Rate_Service', 'type': 'FLOAT64'},
 {'name': 'ERAB_Setup_Success_Rate_All', 'type': 'FLOAT64'},
 {'name': 'Session_Setup_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Session_Abnormal_Release', 'type': 'FLOAT64'},
 {'name': 'Intra_Frequency_Handover_Out_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Radio_Network_Availability_Rate', 'type': 'FLOAT64'},
 {'name': 'Cell_Downlink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'Cell_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_PDCP_User_Throughput', 'type': 'FLOAT64'},
 {'name': 'User_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'UL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'Downlink_Traffic_Vol

In [18]:
def dtype_to_schema(dtype):
    if dtype == "object":
        return "STRING"
    elif dtype == "int64":
        return "INT64"
    elif dtype == "float64":
        return "FLOAT64"
    elif "datetime" in str(dtype):
        return "DATE"
    else:
        return "UNKNOWN"

schema_hourly = [{"name": col, "type": dtype_to_schema(df_hourly[col].dtype)} for col in df_hourly.columns]
schema_hourly

[{'name': 'AREA', 'type': 'STRING'},
 {'name': 'DATE_ID', 'type': 'DATE'},
 {'name': 'HOUR_ID', 'type': 'INT64'},
 {'name': 'ERBS', 'type': 'STRING'},
 {'name': 'EUTRANCELLFDD', 'type': 'STRING'},
 {'name': 'LTE_CSFB_SR', 'type': 'INT64'},
 {'name': 'RRC_Setup_Success_Rate_Service', 'type': 'FLOAT64'},
 {'name': 'ERAB_Setup_Success_Rate_All', 'type': 'FLOAT64'},
 {'name': 'Session_Setup_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Session_Abnormal_Release', 'type': 'FLOAT64'},
 {'name': 'Intra_Frequency_Handover_Out_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Radio_Network_Availability_Rate', 'type': 'FLOAT64'},
 {'name': 'Cell_Downlink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'Cell_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_PDCP_User_Throughput', 'type': 'FLOAT64'},
 {'name': 'User_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'UL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT

In [19]:
with open("encryption_key.key", "rb") as key_file:
    key = key_file.read()

cipher = Fernet(key)

with open("encrypted_credentials.enc", "rb") as encrypted_file:
    encrypted_data = encrypted_file.read()

decrypted_data = cipher.decrypt(encrypted_data)

In [20]:
target_table = "monitoring_396408.tsel_nms"
target_table2 = "monitoring_396408.tsel_nms_hourly"
project_id = "monitoring-396408"
credentials = Credentials.from_service_account_info(eval(decrypted_data.decode()))
job_location = "asia-southeast2"

In [8]:
df.to_gbq(
    destination_table=target_table,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credentials,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [21]:
df_hourly.to_gbq(
    destination_table=target_table2,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credentials,
    table_schema=schema_hourly
)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [22]:
df_hourly.isnull().sum()

AREA                                            0
DATE_ID                                         0
HOUR_ID                                         0
ERBS                                            0
EUTRANCELLFDD                                   0
LTE_CSFB_SR                                     0
RRC_Setup_Success_Rate_Service                  0
ERAB_Setup_Success_Rate_All                     0
Session_Setup_Success_Rate                      0
Session_Abnormal_Release                        0
Intra_Frequency_Handover_Out_Success_Rate       0
Radio_Network_Availability_Rate                 0
Cell_Downlink_Average_Throughput                0
Cell_Uplink_Average_Throughput                  0
DL_PDCP_User_Throughput                         0
User_Uplink_Average_Throughput                  0
DL_Resource_Block_Utilizing_Rate             1206
UL_Resource_Block_Utilizing_Rate             1206
Downlink_Traffic_Volume                         0
Uplink_Traffic_Volume                           0
